In [ ]:
import os 
import openai
import json
from dotenv import load_dotenv

In [ ]:
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
# parameters

TEMPERATURE = 0.0
MAX_TOKENS = 3500
TOP_P = 1.0

In [ ]:
SYSTEM_PROMPT = """

        You are an expert at games of chance / probabilities.

"""

In [ ]:
USER_NAME = "USER"
AI_NAME = "AI EXPERT"
NEW_INTERACTION_DELIMITER = "\n\n"

In [ ]:
def query_gpt(prompt):

    response = openai.Completion.create(
        model = "gpt-4",
        prompt = prompt,
        temperature = TEMPERATURE,
        max_tokens = MAX_TOKENS,
        top_p = TOP_P
    )

    return response.choices[0].text.strip().strip("\n")

In [ ]:
def get_system_prompt(sys_prompt = SYSTEM_PROMPT):

    return [f"SYSTEM PROMPT : {sys_prompt}"]

In [ ]:
def get_conversation(user_input, conversation):

    if not conversation:
        conversation = get_system_prompt()
    user_input_string = f"{USER_NAME} : {user_input}"
    response_trigger = f"{AI_NAME} : "
    conversation.extend([user_input_string, response_trigger])

    return conversation

In [ ]:
def get_response(user_input_string, conversation, use_simple_truncation, verbose):

    conversation = get_conversation(user_input_string, conversation)

    # try to prompt model and then catch if the prompt exceeds the attention window
    first_try = True 
    attention_window_used_up = False

    while first_try or attention_window_used_up:

        flattened_conversation = NEW_INTERACTION_DELIMITER.join(conversation)

        try:
            first_try = False 
            response = query_gpt(flattened_conversation)
            attention_window_used_up = False

        except openai.InvalidRequestError as e:
            attention_window_used_up = True 

            if verbose:
                print("\n*** ATTENTION WINDOW USED UP ***\n")
                print(f"\nERROR : {e}\n")

            if not conversation: 
                return [
                    "ERROR : SYSTEM PROMPT CONSUMING A SIGNIFICANT CHUNK OF WINDOW TOKENS..!"
                ]
            
            if use_simple_truncation:
                conversation = conversation[1:]

            else:
                conversation = conversation[:1] + conversation[2:]

    conversation[-1] += response 

    if verbose:
        print(NEW_INTERACTION_DELIMITER.join(conversation))

    else:
        print(f"{USER_NAME} : {user_input_string}")
        print(f"{AI_NAME} : {response}")

    return conversation

In [ ]:
def chat(user_query, conversation = [], use_simple_truncation = False, verbose = False):
    
    conversation = get_response(user_query, conversation, use_simple_truncation, verbose)

    return conversation

In [ ]:
# TESTING THE CHATBOT

user_inputs = [
    "What card games have the best odds of winning?",
    "what are the odds of winning?",
    "What is the best possible hand that can be dealt in this game?",
    "What is the game with the next best odds?"
]

In [ ]:
conversation = []

verbose = False

simple_truncation = True 

for i, input in enumerate(user_inputs):

    print(f" *** CONVERSATION TURN {i} ***")

    conversation = chat(
        user_inputs, conversation = conversation, use_simple_truncation = simple_truncation, verbose = verbose
    )

    print(conversation)